### Recommendations with MovieTweetings: Most Popular Recommendation

Now that you have created the necessary columns we will be using throughout the rest of the lesson on creating recommendations, let's get started with the first of our recommendations.

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')
del movies['Unnamed: 0']
del reviews['Unnamed: 0']

#### 1. How To Find The Most Popular Movies

For this notebook, we have a single task.  The task is that no matter the user, we need to provide a list of the recommendations based on simply the most popular items.

For this task, we will consider what is "most popular" based on the following criteria:

* A movie with the highest average rating is considered best
* With ties, movies that have more ratings are better
* A movie must have a minimum of 5 ratings to be considered among the best movies
* If movies are tied in their average rating and number of ratings, the ranking is determined by the movie that is the most recent rating

With these criteria, the goal for this notebook is to take a **user_id** and provide back the **n_top** recommendations.  Use the function below as the scaffolding that will be used for all the future recommendations as well.

In [2]:
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [3]:
reviews.head()

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,1,68646,10,1381620027,2013-10-12 23:20:27,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,1,113277,10,1379466669,2013-09-18 01:11:09,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,2,422720,8,1412178746,2014-10-01 15:52:26,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,2,454876,8,1394818630,2014-03-14 17:37:10,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,2,790636,7,1389963947,2014-01-17 13:05:47,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [23]:
reviews.groupby('movie_id').date.max()

movie_id
8          2014-04-08 18:20:11
10         2014-10-09 18:15:53
12         2015-08-10 23:16:19
25         2017-02-27 10:04:59
91         2013-11-23 18:59:55
417        2018-06-26 21:08:46
439        2018-06-24 02:59:09
443        2016-10-23 21:08:32
628        2013-11-29 12:55:28
833        2013-11-29 15:27:27
1223       2013-10-12 17:48:58
1740       2013-11-29 12:54:57
2101       2014-03-24 06:28:08
2130       2014-03-30 04:30:03
2354       2016-02-24 19:41:51
2844       2013-03-09 06:36:12
3740       2018-01-29 03:25:17
3863       2015-05-01 00:03:27
4099       2017-09-09 01:01:51
4100       2017-01-26 19:52:16
4101       2015-04-30 23:49:52
4210       2015-05-01 00:07:39
4395       2015-05-01 00:10:28
4457       2016-05-02 00:03:29
4518       2015-05-04 05:12:52
4546       2015-05-04 05:23:56
4936       2013-07-06 00:21:09
4972       2017-11-06 15:47:21
5074       2015-04-06 20:34:50
5078       2013-08-23 12:56:15
                  ...         
7942708    2018-03-14 22:25:17

In [10]:
def create_ranked_df (movies,reviews):
    average_rating = reviews.groupby('movie_id').rating.mean()
    number_of_rating = reviews.groupby('movie_id').rating.count()
    #latest_rating = reviews.groupby('movie_id').date.max()
    last_rating = pd.DataFrame(reviews.groupby('movie_id').date.max())
    last_rating.columns = ['last_rating']
    
    ratings_count_df=pd.DataFrame({'avg_rating': average_rating,'number_of_rating': number_of_rating })

    movie_recs = pd.merge(ratings_count_df, last_rating, on=['movie_id'])
    
    ranked_movies = movie_recs.sort_values(['avg_rating','number_of_rating','last_rating'], ascending=False)
    
    ranked_movies =ranked_movies[ranked_movies['number_of_rating'] > 4]
    
    return ranked_movies

In [7]:
ranked_movies=pd.merge(ratings_count_df, last_rating, on=['movie_id'])

In [8]:
ranked_movies= ranked_movies.sort_values(['avg_rating','number_of_rating','last_rating'], ascending=False)

In [11]:
ranked_movies.head(15)

,avg_rating,number_of_rating,last_rating
movie_id,,,
4921860,10.0,48,2016-08-14 17:16:50
5262972,10.0,28,2016-01-08 00:44:43
5688932,10.0,14,2018-06-17 01:44:48
2737018,10.0,10,2015-05-10 22:56:01
2560840,10.0,6,2016-01-23 00:30:44
2219210,10.0,6,2013-07-22 23:30:52
4448444,10.0,5,2017-09-09 13:51:48
5131914,10.0,5,2016-04-04 05:52:22
2059318,10.0,5,2014-02-13 15:48:31


In [12]:
create_ranked_df (movies,reviews)

,avg_rating,number_of_rating,last_rating
movie_id,,,
4921860,10.000000,48,2016-08-14 17:16:50
5262972,10.000000,28,2016-01-08 00:44:43
5688932,10.000000,14,2018-06-17 01:44:48
2737018,10.000000,10,2015-05-10 22:56:01
2560840,10.000000,6,2016-01-23 00:30:44
2219210,10.000000,6,2013-07-22 23:30:52
4448444,10.000000,5,2017-09-09 13:51:48
5131914,10.000000,5,2016-04-04 05:52:22
2059318,10.000000,5,2014-02-13 15:48:31


In [16]:
def popular_recommendations(user_id, n_top,ranked_movies):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    # Do stuff
    top_movies = list(ranked_movies['movie_id'][:n_top])
    return top_movies # a list of the n_top movies as recommended

In [25]:
list(ranked_movies[:])

['avg_rating', 'number_of_rating', 'last_rating']

In [19]:
list(ranked_movies['movie_id'][:10])

KeyError: 'movie_id'

Usint the three criteria above, you should be able to put together the above function.  If you feel confident in your solution, check the results of your function against our solution. On the next page, you can see a walkthrough and you can of course get the solution by looking at the solution notebook available in this workspace.  

In [18]:
# Put your solutions for each of the cases here
ranked_movies =create_ranked_df(movies,reviews)
# Top 20 movies recommended for id 1

recs_20_for_1 = popular_recommendations(1,20,ranked_movies)# Your solution list here

# Top 5 movies recommended for id 53968
recs_5_for_53968 = popular_recommendations(53968,5,ranked_movies)# Your solution list here

# Top 100 movies recommended for id 70000
recs_100_for_70000 = popular_recommendations(70000,5,ranked_movies)# Your solution list here

# Top 35 movies recommended for id 43
recs_35_for_43 = popular_recommendations(43,35,ranked_movies)# Your solution list here



KeyError: 'movie_id'

In [ ]:
### You Should Not Need To Modify Anything In This Cell
ranked_movies = t.create_ranked_df(movies, reviews) # only run this once - it is not fast

# check 1 
assert t.popular_recommendations('1', 20, ranked_movies) == recs_20_for_1,  "The first check failed..."
# check 2
assert t.popular_recommendations('53968', 5, ranked_movies) == recs_5_for_53968,  "The second check failed..."
# check 3
assert t.popular_recommendations('70000', 100, ranked_movies) == recs_100_for_70000,  "The third check failed..."
# check 4
assert t.popular_recommendations('43', 35, ranked_movies) == recs_35_for_43,  "The fourth check failed..."

print("If you got here, looks like you are good to go!  Nice job!")

**Notice:** This wasn't the only way we could have determined the "top rated" movies.  You can imagine that in keeping track of trending news or trending social events, you would likely want to create a time window from the current time, and then pull the articles in the most recent time frame.  There are always going to be some subjective decisions to be made.  

If you find that no one is paying any attention to your most popular recommendations, then it might be time to find a new way to recommend, which is what the next parts of the lesson should prepare us to do!


### Part II: Adding Filters

Now that you have created a function to give back the **n_top** movies, let's make it a bit more robust.  Add arguments that will act as filters for the movie **year** and **genre**.  

Use the cells below to adjust your existing function to allow for **year** and **genre** arguments as **lists** of **strings**.  Then your ending results are filtered to only movies within the lists of provided years and genres (as `or` conditions).  If no list is provided, there should be no filter applied.

You can adjust other necessary inputs as necessary to retrieve the final results you are looking for!

Try writing a few tests against the test function in our test function.  Below returns the top 20 movies for user 1 based on the specified year and genre filters.  Does yours return the same? 

```
t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])
```